In [1]:
import numpy as np
import pandas as pd

import os

import tensorflow as tf

from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, SpatialDropout2D, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt

In [2]:
#Training Data and Labels

dim_x = 28
dim_y = 28
batch_size = 32

In [3]:
x_train = pd.read_csv('D:/Academics Canada/AIDI/Semester 2/Capstone 2/Project/MNIST Dataset/sign_mnist_train.csv')
y_train = np.array(x_train['label'])
x_train.drop('label', axis = 1, inplace = True)
x_train = np.array(x_train.values)

In [5]:
print('Data shapes', x_train.shape, y_train.shape, 'classes: ', len(np.unique(y_train)))

Data shapes (27455, 784) (27455,) classes:  24


In [7]:
classes = len(np.unique(y_train))
x_train = x_train.reshape((-1, dim_x, dim_y, 1))
print(np.unique(y_train))

[ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


In [8]:
y = np.zeros((np.shape(y_train)[0], len(np.unique(y_train))))

In [9]:
y_train[y_train>8] = y_train[y_train > 8] - 1

for ii in range(len(y_train)):
    y[ii, y_train[ii]] = 1
y_train = y

In [10]:
no_validation = int(0.1 * (x_train.shape[0]))

x_val = x_train[0:no_validation,...]
y_val = y_train[0:no_validation,...]

x_train = x_train[0:no_validation,...]
y_train = y_train[0:no_validation,...]

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(2745, 28, 28, 1) (2745, 24) (2745, 28, 28, 1) (2745, 24)


In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 10, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 0.1, zoom_range = 0.075)
train_generator = train_datagen.flow(x = x_train, y = y_train, batch_size = batch_size, shuffle = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = test_datagen.flow(x = x_val, y = y_val, batch_size = batch_size, shuffle = True)

In [12]:
model = Sequential()

model.add(Conv2D(filters = 6, kernel_size = (5, 5), strides = 1, input_shape = (dim_x, dim_y, 1), activation = tf.nn.tanh))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Conv2D(filters = 16, kernel_size = (5, 5), strides = 1, activation = tf.nn.tanh))
model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Flatten())
model.add(Dense(120, activation = tf.nn.tanh))
model.add(Dense(84, activation = tf.nn.tanh))
model.add(Dense(classes, activation = tf.nn.softmax))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               30840     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [16]:
model.compile(loss = 'categorical_crossentropy', optimizer = Adam( lr=1e-4), metrics = ['accuracy'])

In [17]:
steps_per_epoch = int(len(y_train) / batch_size)
max_epochs = 500
history = model.fit_generator(generator = train_generator, steps_per_epoch = steps_per_epoch, validation_data = val_generator, validation_steps = 50, epochs = max_epochs, verbose = 2)

W0221 14:04:07.472226 21440 deprecation.py:323] From <ipython-input-17-64bc8a897b89>:3: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.
W0221 14:04:07.603395 21440 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0221 14:04:07.669216 21440 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']


Train for 85 steps, validate for 50 steps
Epoch 1/500
85/85 - 2s - loss: 3.1714 - accuracy: 0.0627 - val_loss: 3.1353 - val_accuracy: 0.0988
Epoch 2/500
85/85 - 1s - loss: 3.1160 - accuracy: 0.1194 - val_loss: 3.0705 - val_accuracy: 0.1437
Epoch 3/500
85/85 - 2s - loss: 3.0352 - accuracy: 0.1589 - val_loss: 2.9486 - val_accuracy: 0.2412
Epoch 4/500
85/85 - 2s - loss: 2.8863 - accuracy: 0.2296 - val_loss: 2.7353 - val_accuracy: 0.2800
Epoch 5/500
85/85 - 1s - loss: 2.6662 - accuracy: 0.2753 - val_loss: 2.4906 - val_accuracy: 0.3119
Epoch 6/500
85/85 - 1s - loss: 2.4600 - accuracy: 0.3111 - val_loss: 2.3005 - val_accuracy: 0.3525
Epoch 7/500
85/85 - 1s - loss: 2.3046 - accuracy: 0.3550 - val_loss: 2.1218 - val_accuracy: 0.4006
Epoch 8/500
85/85 - 1s - loss: 2.1618 - accuracy: 0.3907 - val_loss: 2.0015 - val_accuracy: 0.4550
Epoch 9/500
85/85 - 1s - loss: 2.0517 - accuracy: 0.4102 - val_loss: 1.8826 - val_accuracy: 0.4919
Epoch 10/500
85/85 - 1s - loss: 1.9679 - accuracy: 0.4346 - val_los